In [0]:
import pandas as pd
import numpy as np

In [0]:
df = {}

In [0]:
fn = '/datascope/subaru/data/targeting/MW/outerdisk_l180_b22_ENG/ga_targets_outerdisk_l180_b22_ENG-v0.csv'
x = pd.read_csv(fn)

df['gaia'] = x[~x['g_gaia'].isna()]
df['ps1'] = x[~x['g_ps1'].isna()]

df['gaia'], df['ps1']

In [0]:
fn = '/datascope/subaru/data/targeting/MW/VMP/jplus_field_111_38.csv'
df['jplus'] = pd.read_csv(fn)
df['jplus']

In [0]:
fn = '/datascope/subaru/data/targeting/MW/VMP/Li_Subaru_LAMOST_field_111_38.csv'
df['lamost'] = pd.read_csv(fn)
df['lamost']

In [0]:
# X-match code from Miho

import astropy.coordinates as coord
import astropy.units as u
from astropy.table import Table, vstack
from astropy.coordinates import SkyCoord, match_coordinates_sky

def remove_duplicate(tb, depth):

    # Inputs: 
    #     tb:  Astropy Table format 
    #     depth: "bright" or "faint"
    
    print("Initial number of table entries: ", len(tb))

    # Two iterations are needed to deal with duplicates among up to three catalogs.
    for i in range(0, 2):

        print("Length of the input table:", len(tb))
        
        coords = SkyCoord(ra = tb['ra'] * u.degree, dec = tb['dec'] * u.degree)

        discards = np.array([], dtype = bool)
        
        for row in tb:
            discard = False
            objid = row['obj_id']
            coord = SkyCoord(ra = row['ra'] * u.degree, dec = row['dec'] * u.degree)
            filt = (tb['obj_id'] == objid) & (tb['input_catalogs'] == row['input_catalogs']) # Object itself
            coords_filt = coords[~filt]          # Remove the object itself
            
            idx, d2d, d3d = match_coordinates_sky(coord, coords_filt, nthneighbor=1)
            
            if d2d.value < 2./3600.: 
                
                if depth == "faint":
                    if i == 0 and row['input_catalogs'] == "PS1":
                        discard = True
                    elif i == 1 and (row['input_catalogs'] == "Gaia"):
                        discard = True
                elif depth == "bright":
                    if i == 0 and (row['input_catalogs'] == "SEGUE" or \
                                   row['input_catalogs'] == "DESI" or row['input_catalogs'] == "PS1"):                       
                        discard = True
                    elif i ==1 and (row['input_catalogs'] == "Pristine-Gaia" or row['input_catalogs'] == "PristineDR1"):
                        discard = True
                    
            discards = np.hstack((discards, discard))
        
        tb = tb[~discards]

        print("After the iteration %i, the number of entries: %i"%(i, len(tb)))
    return(tb)

In [0]:
from astropy.coordinates import SkyCoord, match_coordinates_sky
import astropy.coordinates as coord
import astropy.units as u

# Use astropy to cross-match the catalogs df['ps1'] and df['gaia']
ps1_coords = SkyCoord(ra=np.array(df['ps1']['ra']) * u.degree, dec=np.array(df['ps1']['dec']) * u.degree)
gaia_coords = SkyCoord(ra=np.array(df['gaia']['ra']) * u.degree, dec=np.array(df['gaia']['dec']) * u.degree)

ps1_coords, gaia_coords

In [0]:
idx, sep2d, dist3d = match_coordinates_sky(ps1_coords, gaia_coords, nthneighbor=1)
# idx: indices into gaia_coords
# sep2d: on-sky separation, one entry for each ps1_coords

In [0]:
idx.shape, sep2d.shape, ps1_coords.shape

In [0]:
sep2d.to(u.arcsec).value